In [101]:
import pandas as pd
from microdf import MicroDataFrame

cols = ['SSUID','PNUM','MONTHCODE','ERESIDENCEID','ERELRPE','SPANEL','SWAVE',
      
      'WPFINWGT',
      
      'ESEX','TAGE','TAGE_EHC','ERACE','EORIGIN','EEDUC', 'EDEPCLM', 'EMS', 'EFSTATUS',
      
      'TJB1_TXAMT', 'TJB1_MSUM', 'TJB1_OCC', 'TJB1_IND', 'AJB1_TXAMT', 'EJB1_TYPPAY3',
      'TJB2_TXAMT', 'TJB2_MSUM', 'TJB2_OCC', 'TJB2_IND', 'AJB2_TXAMT', 'EJB2_TYPPAY3',
      'TJB3_TXAMT', 'TJB3_MSUM', 'TJB3_OCC', 'TJB3_IND', 'AJB3_TXAMT', 'EJB3_TYPPAY3',
      'TJB4_TXAMT', 'TJB4_MSUM', 'TJB4_OCC', 'TJB4_IND', 'AJB4_TXAMT', 'EJB4_TYPPAY3',
      
      'TPTOTINC']

for col in cols:
    if "JB1" in col:
        for i in range(2, 8):
            cols.append(col.replace("JB1", f"JB{i}"))

df = pd.read_csv("~/Downloads/pu2023.csv", delimiter="|", usecols=cols)

df["total_earnings"] = df.TPTOTINC
df["total_tips"] = df[df.columns[df.columns.str.contains("TXAMT")]].fillna(0).sum(axis=1)

df["is_under_18"] = (df.TAGE < 18) & (df.MONTHCODE == 12)
df["is_under_6"] = (df.TAGE < 6) & (df.MONTHCODE == 12)
df["count_under_18"] = df.groupby("SSUID")["is_under_18"].sum().loc[df.SSUID.values].values
df["count_under_6"] = df.groupby("SSUID")["is_under_6"].sum().loc[df.SSUID.values].values
df["household_weight"] = df.WPFINWGT
df["household_id"] = df.SSUID

df = MicroDataFrame(df[["household_id", "total_earnings", "total_tips", "count_under_18", "count_under_6", "household_weight"]], weights="household_weight")

In [103]:
households = MicroDataFrame(pd.DataFrame(df).groupby("household_id").sum(), weights="household_weight")

In [106]:
import plotly.express as px
from policyengine_core.charts import *

fig = px.bar(
    (households.total_tips > 0).groupby(households.total_earnings.decile_rank()).mean()
).update_layout(
    title="Percent of households with tips by household earnings decile, SIPP"
)

format_fig(fig)